In [1]:
from cameo import load_model
from os import getcwd
from Functions_Modules import curation_tools as ct
from math import isnan
from numpy import zeros

/Users/clie/.virtualenvs/modeling/lib/python2.7/site-packages/cobra/io/sbml3.py:24: UserWarning: Install lxml for faster SBML I/O
  warn("Install lxml for faster SBML I/O")


In [2]:
relative_directory = getcwd()
model = load_model(relative_directory + '/Reconstructions/MethylococcusModel11.json')

In [3]:
len(model.reactions.BIOMASS_REACTION.metabolites)

92

In [4]:
# import escher
        
# maps = escher.Builder(
#                     model=model,
#                     map_name = 'Methylococcus capsulatus genome-scale metabolic model',
#                     map_json='/Users/clie/Desktop/EFPro2/Reconstructions/Metabolic Map/15MinorChanges.json',
#                     show_gene_reaction_rules=True,
#                     use_3d_transform = True
#                     )

# maps.set_reaction_scale([ { 'type': 'min', 'color': 'rgb(235,89,162)', 'size': 12 },
#    { 'type': 'median', 'color': 'rgb(200,200,200)', 'size': 20 },
#    { 'type': 'max', 'color': 'rgb(255,255,109)', 'size': 25 } ])
# maps.display_in_browser()

# Last Check-ups

## Observing the Default State of the Model

In [5]:
print model.medium

     reaction_id       reaction_name  lower_bound  upper_bound
0        EX_pi_c  phosphate exchange      -1000.0       1000.0
1        EX_o2_c   dioxygen exchange      -1000.0       1000.0
2       EX_nh3_c     NH4(+) exchange      -1000.0       1000.0
3       EX_so4_c    sulfate exchange      -1000.0       1000.0
4        EX_cl_c      Cl(-) exchange      -1000.0       1000.0
5       EX_na1_c      Na(+) exchange         -1.0       1000.0
6         EX_k_c       K(+) exchange      -1000.0       1000.0
7       EX_mg2_c     Mg(2+) exchange      -1000.0       1000.0
8       EX_ca2_c     Ca(2+) exchange      -1000.0       1000.0
9       EX_ch4_p    methane exchange      -1000.0         -0.1
10  EX_cobalt2_c       Co2+ exchange      -1000.0       1000.0
11       EX_cu_c        Cu+ exchange      -1000.0       1000.0


In [6]:
print model.objective

Maximize
-1.0*BIOMASS_REACTION_reverse_42726 + 1.0*BIOMASS_REACTION


Specic growth rate in default conditions (1/h)

In [7]:
max_growth = model.solve().f
print max_growth

7.04946219417


In [8]:
model.reactions.EX_ch4_p.lower_bound = - 18.46

In [9]:
max_growth = model.solve().f
print max_growth

0.260266144209


Specic growth rate of E. coli in LB-medium (1/h)

In [10]:
import math

In [11]:
math.log(2)/0.5

1.3862943611198906

Doubling time of the model in default conditions (in minutes)

In [12]:
(math.log(2)/max_growth) * 60

159.7934720247074

## Double check reaction and formula identity with BiGG Database

In [13]:
import pandas as pd

In [14]:
def bigg_web_query(url,filename):
    from requests import get
    import json
    
    try: 
        response = get(url)
        response.ok
        
        with open(filename, 'w') as f:
            json.dump(response.json()['results'],f)
    except Exception as e:
        print(e)
    
    return 

In [15]:
url = 'http://bigg.ucsd.edu/api/v2/universal/reactions'
filename = 'bigg_universal_reaction_dump.json'

In [16]:
bigg_web_query(url,filename)

In [17]:
bigg_uni_rxn_dump = pd.read_json('bigg_universal_reaction_dump.json',orient='column')

In [18]:
bigg_uni_rxn_dump_dict = bigg_uni_rxn_dump.set_index('bigg_id').to_dict('index')

Identify reaction IDs that match BiGG standards and those that don't

In [19]:
def ident_BiGG_rxn_id_matches(model_reactions,database_dict):
    BiGG_ID_y_n = []
    temp_storage = ''

    for i,rxn in enumerate(model_reactions):
        if not rxn.id.startswith('EX_'):
            if '_' in rxn.id:
                temp_storage = rxn.id.rpartition('_')[0] 
            else:
                temp_storage = rxn.id
            if temp_storage in database_dict.keys():
                BiGG_ID_y_n.append('matched')
                # TODO: Pull all annotations
                # HERE: Sort them into the model
            else:
                BiGG_ID_y_n.append(rxn.id)
    return BiGG_ID_y_n


In [20]:
BiGG_ID_y_n = ident_BiGG_rxn_id_matches(model.reactions,bigg_uni_rxn_dump_dict)

non_bigg_ids = filter(lambda x: x != 'matched', BiGG_ID_y_n)

len(non_bigg_ids)

132

In [21]:
BiGG_ID_y_n

['matched',
 u'ALCD2if_c',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 u'G6PDH2_c',
 'matched',
 'matched',
 'matched',
 u'PKL_c',
 u'HPS_c',
 u'PHI_c',
 u'GALU_c',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 'matched',
 '

In [22]:
for rxn in model.reactions:
    if not (rxn.id.startswith('EX_') or rxn.id.startswith('DM_')):
        if rxn.id.endswith('_p'):
            rxn.id = rxn.id.rpartition('_')[0] + 'pp'
        if rxn.id.endswith('_im'):
            rxn.id = rxn.id.rpartition('_')[0] + 'pp'
        if rxn.id.endswith('_c'):
            rxn.id = rxn.id.rpartition('_')[0]
        if rxn.id.endswith('_e'):
            rxn.id = rxn.id.rpartition('_')[0] + 'e'
        if rxn.id.endswith('_Gapfilling'):
            rxn.id = rxn.id.rpartition('_')[0] + '_GAPFILLING'
model.solver.update()

In [23]:
BiGG_ID_y_n = ident_BiGG_rxn_id_matches(model.reactions,bigg_uni_rxn_dump_dict)

non_bigg_ids = filter(lambda x: x != 'matched', BiGG_ID_y_n)

len(non_bigg_ids)

99

In [24]:
for rxnid in non_bigg_ids:
    if '__' in rxnid:
        model.reactions.get_by_id(rxnid).id = model.reactions.get_by_id(rxnid).id.replace('__','_')

In [25]:
BiGG_ID_y_n = ident_BiGG_rxn_id_matches(model.reactions,bigg_uni_rxn_dump_dict)

non_bigg_ids = filter(lambda x: x != 'matched', BiGG_ID_y_n)

len(non_bigg_ids)

97

In [26]:
for rxn in non_bigg_ids[2:]:
    if not model.reactions.get_by_id(rxn).reversibility:
        if not rxn.endswith('i'):
            model.reactions.get_by_id(rxn).id = model.reactions.get_by_id(rxn).id + 'i'

In [27]:
BiGG_ID_y_n = ident_BiGG_rxn_id_matches(model.reactions,bigg_uni_rxn_dump_dict)

non_bigg_ids = filter(lambda x: x != 'matched', BiGG_ID_y_n)

len(non_bigg_ids)

89

In [28]:
model.reactions.ALCD2if.id = 'ALCD2ir'

'ALCD2ir' is the id in the BiGG Database, but the reaction is irreversible in all three models and hence should not bear the 'r' suffix which implies it is reversible.

In [29]:
model.reactions.G6PDH2 = 'G6PDH2i'

'G6PDH2r' is the id in the Universal BiGG Database. It is reversible in many of the Ecoli models, but not in the iND750 yeast model. I am making the decision to keep this one irreversible (in order to keep the model functional as it is right now), but this should probably be looked at later!

In [30]:
model.reactions.PHI.id = 'AH6PI'

In [31]:
model.reactions.PRAGSi

Id,PRAGSi
Name,1 H(+) + 1 N(1)-(5-phospho-D-ribosyl)glycinamide(1-) + 1 ADP + 1 phosphate = 1 glycine + 1 ATP + 1 5-phosphonato-D-ribosylaminium(1-)
Stoichiometry,atp_c + gly_c + pram_c --> adp_c + gar_c + h_c + pi_c
GPR,MCA0361
Lower bound,0.000000
Upper bound,1000.000000


'PRAGSr' is the id in the Universal BiGG Database. It is reversible in many of the Ecoli models. I am making the decision to keep this one irreversible (in order to keep the model functional as it is right now), but this should probably be looked at later!

In [32]:
model.reactions.ADSL2.id = 'ADSL2r'

In [33]:
model.reactions.ADSL1i

Id,ADSL1i
Name,"1 AMP + 1 fumarate(2-) = 1 N(6)-(1,2-dicarboxylatoethyl)-AMP(4-)"
Stoichiometry,dcamp_c --> amp_c + fum_c
GPR,MCA1784
Lower bound,0.000000
Upper bound,1000.000000


'ADSL1r' is the id in the Universal BiGG Database. It is reversible in many of the Ecoli models. I am making the decision to keep this one irreversible (in order to keep the model functional as it is right now), but this should probably be looked at later!

In [34]:
model.reactions.AGT.id = 'AGTi'

This is reversible as a Universal reaction but irreversible in yeast and synechocystis.

In [35]:
model.reactions.ALAD_Li.id = 'ALAD_L'

'ALAD_L' is the ID in the Universal BiGG Database, however in each of the 5 models this reaction is actually irreversible (but missing the i in the ID).

In [36]:
model.reactions.GHMT2.id = 'GHMT2r'

In [37]:
model.reactions.SERD_Li.id = 'SERD_L'

In [38]:
model.reactions.SHK3Di

Id,SHK3Di
Name,1 H(+) + 1 NADPH + 1 3-Dehydroshikimate = 1 NADP(+) + 1 shikimate
Stoichiometry,3dhsk_c + h_c + nadph_c --> nadp_c + skm_c
GPR,MCA2794
Lower bound,0.000000
Upper bound,1000.000000


'SHK3Dr' in BiGG Universal. It is reversible in many of the Ecoli models. I am making the decision to keep this one irreversible (in order to keep the model functional as it is right now), but this should probably be looked at later!

In [39]:
model.reactions.DHQDi.id = 'DQDHi'

I am making the decision to keep this one irreversible (in order to keep the model functional as it is right now), but this should probably be looked at later!

In [40]:
model.reactions.LDH_Di

Id,LDH_Di
Name,1 H(+) + 1 NADH(2-) + 1 pyruvate = 1 (R)-lactate + 1 NAD(+)
Stoichiometry,h_c + nadh_c + pyr_c --> lac_D_c + nad_c
GPR,MCA0182
Lower bound,0.000000
Upper bound,1000.000000


'LDH_D' in BiGG Universal and many of the models and reversible accordingly. Keeping it as it is for now, should be looked at later.

In [41]:
model.reactions.MICITDi

Id,MICITDi
Name,"1 (2S,3R)-3-hydroxybutane-1,2,3-tricarboxylate = 1 (Z)-but-2-ene-1,2,3-tricarboxylate + 1 H2O"
Stoichiometry,micit_c --> 2mcacn_c + h2o_c
GPR,MCA2485
Lower bound,0.000000
Upper bound,1000.000000


'MICITDr' in BiGG Universal and many of the models and reversible accordingly. Keeping it as it is for now, should be looked at later.

In [42]:
model.reactions.NAD_H2i.id = 'NAD_H2'

'NAD_H2' in and reversible BiGG Universal, but in the only other model that of Geobacter metallireducens iAF987 it is irreversible. So, I'm keeping it like this.

In [43]:
model.reactions.FMFRDi.id = 'FMFRDHi'

In [44]:
model.reactions.sMMOi.id = 'SMMOi'

In [45]:
model.reactions.ACP1_FMNi.id = 'ACP1_FMN'

'ACP1_FMN' in and reversible BiGG Universal, but in the other models it is irreversible. So, I'm keeping it like this, even if it should have an 'i' tag.

In [46]:
model.reactions.NNATi

Id,NNATi
Name,1 H(+) + 1 nicotinate beta-D-ribonucleotide + 1 ATP = 1 diphosphate(3-) + 1 deamido-NAD(2-)
Stoichiometry,atp_c + h_c + nicrnt_c --> dnad_c + ppi_c
GPR,MCA1879
Lower bound,0.000000
Upper bound,1000.000000


'NNATr' is the id in the Universal BiGG Database. It is reversible in many of models. I am making the decision to keep this one irreversible (in order to keep the model functional as it is right now), but this should probably be looked at later!

In [47]:
model.reactions.NORppi.id = 'NORZ2pp'

Similar to NORZpp but using Cytochrome C instead.

In [48]:
print model.solve().f

0.260266144209


In [49]:
model.remove_reactions([model.reactions.PAH])

In [50]:
model.reactions.GAPDH_nadp_hi.id = 'GAPDH_nadp'

In [51]:
model.reactions.VPAMTi

Id,VPAMTi
Name,1 3-methyl-2-oxobutanoate + 1 L-alanine zwitterion = 1 L-valine zwitterion + 1 pyruvate
Stoichiometry,3mob_c + ala__L_c --> pyr_c + val__L_c
GPR,MCA2344
Lower bound,0.000000
Upper bound,1000.000000


'VPAMTr' is the id in the Universal BiGG Database. It is reversible in many of models. I am making the decision to keep this one irreversible (in order to keep the model functional as it is right now), but this should probably be looked at later!

In [52]:
model.reactions.DHORD_NADi

Id,DHORD_NADi
Name,1 H(+) + 1 NADH(2-) + 1 orotate = 1 (S)-dihydroorotate + 1 NAD(+)
Stoichiometry,dhor_S_c + nad_c --> h_c + nadh_c + orot_c
GPR,MCA0768 or MCA1242
Lower bound,0.000000
Upper bound,1000.000000


'DHORD_NAD' is the id in the Universal BiGG Database. It is reversible in many of models. I am making the decision to keep this one irreversible (in order to keep the model functional as it is right now), but this should probably be looked at later!

In [53]:
model.genes.MCA1660.reactions

frozenset({<Reaction AHMTi at 0x113a92390>,
           <Reaction SGAT at 0x113e00410>,
           <Reaction GHMT2r at 0x113e00c10>,
           <Reaction GCCb at 0x113e11850>})

In [54]:
model.reactions.BIOMASS_REACTIONi.id = 'BIOMASS_Mcapsulatus'

In [55]:
model.reactions.pMMOppi.id = 'PMMOipp'

In [56]:
model.reactions.NADH5ppi = 'NADH5'

In [57]:
model.reactions.MEOHDH_cytppi.id = 'MEOHDHipp'

In [58]:
model.reactions.MEOHtpp.name = 'Methanol reversible transport via diffusion (periplasm)'

In [59]:
model.reactions.MEOHtpp.id = 'MEOHtrpp'

In [60]:
model.reactions.CH4tpp.id = 'CH4trpp'

In [61]:
model.reactions.FALDDHppi.id = 'FALDDHipp'

In [62]:
model.reactions.CyLCyHOxpp.id = 'CyLCyHORpp'

In [63]:
model.reactions.CYTCBDppi.id = 'CYTCBDipp'

In [64]:
model.reactions.CYTAA3ppi.id = 'CYTAA3ipp'

In [65]:
model.reactions.ATPShppi.id = 'ATPSipp'

In [66]:
model.reactions.ATPSNappi.id = 'ATPSNaipp'

In [67]:
model.reactions.PPAptppi.id = 'PPA3ipp'

In [68]:
model.reactions.NQRppi.id = 'NADHNQR2ipp'

In [69]:
model.reactions.h2otpp.id = 'H2Otpp'

In [70]:
model.reactions.o2tpp.id = 'O2tpp'

In [71]:
model.reactions.FeoBtpp.gene_reaction_rule = 'MCA1097'

In [72]:
model.reactions.DES_9_2_Modifiedi.id = 'DES_9i_Modified'

In [73]:
model.reactions.get_by_id('3OAR150_GAPFILLINGi').id = '3OAR150_Lumped'

In [74]:
model.reactions.get_by_id('3OAR170_GAPFILLINGi').id = '3OAR170_Lumped'

In [75]:
model.reactions.CFACPOA2H_Lumpedi.id = 'CFACPOA2H_Lumped'

In [76]:
model.reactions.OBDHi.id = 'OBDHc'

This reaction is tagged with a 'c' which it technically shouldn't need. It is also reversible in the Universal reaction Database but not in any of the models uploaded.

In [77]:
model.reactions.DM_4hba_ci.id = 'DM_4hba_c'

In [78]:
model.reactions.CLPNS_MCi.id = 'CLPNS_MC'

In [79]:
model.reactions.MC_AFAAi.id = 'MC_AFAA'

In [80]:
model.reactions.DM_doxopa_ci.id = 'DM_doxopa_c'

In [81]:
model.reactions.AMOsi.id = 'SAMMOi'

In [82]:
model.reactions.AMOpppi.id = 'PAMMOipp'

In [83]:
model.reactions.NH3pp.id = 'NH3tpp'

In [84]:
model.reactions.HAMpp.id = 'HAMtpp'

In [85]:
model.reactions.NO2pp.id = 'NO2tpp'

In [86]:
model.reactions.CYP460ppi.id = 'CYP460ipp'

In [87]:
model.reactions.HAOppi.id = 'HAORipp'

In [88]:
model.reactions.no3t_omi.id = 'NO3tex'

In [89]:
model.reactions.NO3abcppppi.id = 'NO3abcpp'

In [90]:
model.reactions.NO3t7pppp.id = 'NO3t7pp'

In [91]:
model.reactions.ALAD_L

Id,ALAD_L
Name,1 H2O + 1 L-alanine zwitterion + 1 NAD(+) = 1 H(+) + 1 NADH(2-) + 1 NH4(+) + 1 pyruvate
Stoichiometry,h_c + nadh_c + nh3_c + pyr_c --> ala__L_c + h2o_c + nad_c
GPR,MCA0563 or MCA1406
Lower bound,0.000000
Upper bound,1000.000000


In [92]:
model.remove_reactions([model.reactions.ALAD_Lyi])

In [93]:
model.reactions.pMMO_dir_couplppi.name = 'particulate Methane Monooxygenase - Direct Coupling' 
model.reactions.pMMO_dir_couplppi.id = 'PMMODCipp'

In [94]:
BiGG_ID_y_n = ident_BiGG_rxn_id_matches(model.reactions,bigg_uni_rxn_dump_dict)

non_bigg_ids = filter(lambda x: x != 'matched', BiGG_ID_y_n)

len(non_bigg_ids)

75

In [95]:
len([x for x in model.metabolites if x.formula == ''])

7

In [96]:
[x for x in model.metabolites if x.formula == '']

[<Metabolite dna_c at 0x113b3a4d0>,
 <Metabolite dnac_c at 0x113af7b10>,
 <Metabolite pepd_c at 0x113a9a890>,
 <Metabolite sertrna_sec__c at 0x11402ab50>,
 <Metabolite focytb5_c at 0x11402aed0>,
 <Metabolite ficytb5_c at 0x11402af50>,
 <Metabolite no3_e at 0x114091e50>]

In [97]:
model.metabolites.dnac_c.formula = 'X'

In [98]:
model.metabolites.dna5mtc_c.formula = 'XCH2'

In [99]:
model.remove_reactions([model.reactions.EX_dna_c,
                        model.reactions.EX_pepd_c,
                        model.reactions.EX_sertrna_sec__c,
                        model.reactions.EX_focytb5_c,
                        model.reactions.EX_ficytb5_c],remove_orphans=True)

In [100]:
model.metabolites.no3_e.formula = 'NO3'

In [101]:
len([x for x in model.metabolites if x.formula == ''])

0

In [102]:
len(model.reactions)

1466

In [103]:
len(non_bigg_ids)

75

In [104]:
len(model.genes)

657

In [105]:
len(model.essential_genes())

178

In [106]:
len(model.essential_reactions())

334

In [107]:
from cameo import flux_variability_analysis

cameo_model_fva = model.copy()

for rxn in cameo_model_fva.reactions:
    if rxn.id.startswith("DM_") or  rxn.id.startswith("EX_"):
        rxn.upper_bound = 1000
        rxn.lower_bound = -1000

In [108]:
fva_result = flux_variability_analysis(cameo_model_fva, reactions=cameo_model_fva.reactions)

In [109]:
blocked_reactions = fva_result.data_frame.query('lower_bound == 0 & upper_bound == 0')

In [110]:
len(blocked_reactions)

186

In [111]:
len([x for x in model.reactions if not x.id.startswith('EX_') and not x.id.startswith('DM_')])

746

In [112]:
import pytest

In [113]:
model.objective = model.reactions.EX_atp_c

In [114]:
model.reactions.EX_atp_c

Id,EX_atp_c
Name,ATP exchange
Stoichiometry,atp_c -->
GPR,
Lower bound,0.000000
Upper bound,1000.000000


In [115]:
name_dict = {}
for rxn in model.reactions:
    if '=' in rxn.name:
        if rxn.id in bigg_uni_rxn_dump_dict:
            if 'name' in bigg_uni_rxn_dump_dict[rxn.id]:
                name_dict[rxn.id] = bigg_uni_rxn_dump_dict[rxn.id]['name']

In [116]:
len(name_dict)

574

In [117]:
model.reactions.FeoBtpp.id = 'FE2tpp'

In [118]:
for rxn in model.reactions:
    if rxn.id in name_dict:
        print rxn, name_dict[rxn.id]
        model.reactions.get_by_id(rxn.id).name = name_dict[rxn.id] 

FBA: fdp_c <=> dhap_c + g3p_c Fructose-bisphosphate aldolase
ALCD2ir: etoh_c + nad_c --> acald_c + h_c + nadh_c Alcohol dehydrogenase  reverse rxn  acetaldehyde   ethanol 
PDHam1hi: h_c + pyr_c + thmpp_c --> 2ahethmpp_c + co2_c Pyruvate dehydrogenase (acetyl-transferring)
PYK: adp_c + h_c + pep_c --> atp_c + pyr_c Pyruvate kinase
ENO: 2pg_c <=> h2o_c + pep_c Enolase
PGM: 3pg_c <=> 2pg_c Phosphoglycerate mutase
GAPD: g3p_c + nad_c + pi_c --> 13dpg_c + h_c + nadh_c Glyceraldehyde-3-phosphate dehydrogenase
TPI: dhap_c <=> g3p_c Triose-phosphate isomerase
PGIA: f6p_c <=> g6p_A_c Glucose-6-phosphate isomerase (g6p-A)
PGCM: g6p_A_c <=> g1p_c Phosphoglucomutase
PGI: f6p_c <=> g6p_c Glucose-6-phosphate isomerase
G6PI_1: g6p_A_c <=> g6p_c Glucose-6-phosphate isomerase
GLUKA: Glc_aD_c + atp_c <=> adp_c + g6p_A_c + h_c Glucokinase/hexokinase (glc-A)
PGK: 13dpg_c + adp_c --> 3pg_c + atp_c Phosphoglycerate kinase
ACS: ac_c + atp_c + coa_c --> accoa_c + amp_c + ppi_c Acetyl-CoA synthetase
SUCOAS: at

## Reaction Annotations

In [119]:
filename_ecoli = '/Users/clie/Downloads/iJO1366.xml'

In [120]:
ecoli = load_model(filename_ecoli)

In [121]:
rxns_without_annotations = [rxn for rxn in model.reactions if rxn.annotation == {} and 'EX_' not in rxn.id]

In [122]:
len(rxns_without_annotations)

88

In [123]:
for rxn in rxns_without_annotations:
    source = ''
    
    if '_GAPFILLING' in rxn.id:
        query_id = rxn.id.split('_')[0]
    else:
        query_id = rxn.id
    
    try:
        source = ecoli.reactions.get_by_id(query_id)
    except:
        print 'Could not find the following reaction ID in iJO2366 {}'.format(query_id)
        
    if source != '' and source.annotation != {}:
            for key in source.annotation:
                if key not in rxn.annotation:
                    rxn.annotation[key] = source.annotation[key]
    else:
        print 'The source annotation is an empty dictionary'


Could not find the following reaction ID in iJO2366 BIOMASS_Mcapsulatus
The source annotation is an empty dictionary
Could not find the following reaction ID in iJO2366 CH4trpp
The source annotation is an empty dictionary
Could not find the following reaction ID in iJO2366 OCT
The source annotation is an empty dictionary
Could not find the following reaction ID in iJO2366 FACOAL180
The source annotation is an empty dictionary
Could not find the following reaction ID in iJO2366 FACOAL140
The source annotation is an empty dictionary
Could not find the following reaction ID in iJO2366 DES_9i_Modified
The source annotation is an empty dictionary
Could not find the following reaction ID in iJO2366 FA180ACPHi
The source annotation is an empty dictionary
Could not find the following reaction ID in iJO2366 3OAR150_Lumped
The source annotation is an empty dictionary
Could not find the following reaction ID in iJO2366 3OAR170_Lumped
The source annotation is an empty dictionary
Could not find the

In [124]:
rxns_without_annotations = [rxn for rxn in model.reactions if rxn.annotation == {} and 'EX_' not in rxn.id]

In [125]:
len(rxns_without_annotations)

35

In [126]:
clamy_file = '/Users/clie/Downloads/iRC1080.xml'
clamydomonas = load_model(clamy_file)

In [127]:
for rxn in rxns_without_annotations:
    source = ''
    
    if '_GAPFILLING' in rxn.id:
        query_id = rxn.id.split('_')[0]
    else:
        query_id = rxn.id
    
    try:
        source = clamydomonas.reactions.get_by_id(query_id)
    except:
        print 'Could not find the following reaction ID in iRC1080 {}'.format(query_id)
        
    if source != '' and source.annotation != {}:
            for key in source.annotation:
                if key not in rxn.annotation:
                    rxn.annotation[key] = source.annotation[key]
    else:
        print 'The source annotation is an empty dictionary'


Could not find the following reaction ID in iRC1080 BIOMASS_Mcapsulatus
The source annotation is an empty dictionary
Could not find the following reaction ID in iRC1080 CH4trpp
The source annotation is an empty dictionary
Could not find the following reaction ID in iRC1080 FACOAL180
The source annotation is an empty dictionary
Could not find the following reaction ID in iRC1080 FACOAL140
The source annotation is an empty dictionary
Could not find the following reaction ID in iRC1080 DES_9i_Modified
The source annotation is an empty dictionary
Could not find the following reaction ID in iRC1080 3OAR150_Lumped
The source annotation is an empty dictionary
Could not find the following reaction ID in iRC1080 3OAR170_Lumped
The source annotation is an empty dictionary
Could not find the following reaction ID in iRC1080 FA150ACPHi
The source annotation is an empty dictionary
Could not find the following reaction ID in iRC1080 FA170ACPHi
The source annotation is an empty dictionary
Could not f

In [128]:
rxns_without_annotations = [rxn for rxn in model.reactions if rxn.annotation == {} and 'EX_' not in rxn.id]
len(rxns_without_annotations)

31

In [129]:
synechocystis_file = '/Users/clie/Downloads/iJN678.xml'
synechocystis = load_model(synechocystis_file)

In [130]:
for rxn in rxns_without_annotations:
    source = ''
    
    if '_GAPFILLING' in rxn.id:
        query_id = rxn.id.split('_')[0]
    else:
        query_id = rxn.id
    
    try:
        source = synechocystis.reactions.get_by_id(query_id)
    except:
        print 'Could not find the following reaction ID in iJN678 {}'.format(query_id)
        
    if source != '' and source.annotation != {}:
            for key in source.annotation:
                if key not in rxn.annotation:
                    rxn.annotation[key] = source.annotation[key]
    else:
        print 'The source annotation is an empty dictionary'


Could not find the following reaction ID in iJN678 BIOMASS_Mcapsulatus
The source annotation is an empty dictionary
Could not find the following reaction ID in iJN678 CH4trpp
The source annotation is an empty dictionary
Could not find the following reaction ID in iJN678 FACOAL180
The source annotation is an empty dictionary
Could not find the following reaction ID in iJN678 FACOAL140
The source annotation is an empty dictionary
Could not find the following reaction ID in iJN678 DES_9i_Modified
The source annotation is an empty dictionary
Could not find the following reaction ID in iJN678 3OAR150_Lumped
The source annotation is an empty dictionary
Could not find the following reaction ID in iJN678 3OAR170_Lumped
The source annotation is an empty dictionary
Could not find the following reaction ID in iJN678 FA150ACPHi
The source annotation is an empty dictionary
Could not find the following reaction ID in iJN678 FA170ACPHi
The source annotation is an empty dictionary
Could not find the f

In [131]:
rxns_without_annotations = [rxn for rxn in model.reactions if rxn.annotation == {} and 'EX_' not in rxn.id]
len(rxns_without_annotations)

27

In [132]:
ljungdahlii_file = '/Users/clie/Downloads/iHN637.xml'
ljungdahlii = load_model(ljungdahlii_file)

In [133]:
for rxn in rxns_without_annotations:
    source = ''
    
    if '_GAPFILLING' in rxn.id:
        query_id = rxn.id.split('_')[0]
    else:
        query_id = rxn.id
    
    try:
        source = ljungdahlii.reactions.get_by_id(query_id)
    except:
        print 'Could not find the following reaction ID in iHN637 {}'.format(query_id)
        
    if source != '' and source.annotation != {}:
            for key in source.annotation:
                if key not in rxn.annotation:
                    rxn.annotation[key] = source.annotation[key]
    else:
        print 'The source annotation is an empty dictionary'


Could not find the following reaction ID in iHN637 BIOMASS_Mcapsulatus
The source annotation is an empty dictionary
Could not find the following reaction ID in iHN637 CH4trpp
The source annotation is an empty dictionary
Could not find the following reaction ID in iHN637 DES_9i_Modified
The source annotation is an empty dictionary
Could not find the following reaction ID in iHN637 3OAR150_Lumped
The source annotation is an empty dictionary
Could not find the following reaction ID in iHN637 3OAR170_Lumped
The source annotation is an empty dictionary
Could not find the following reaction ID in iHN637 FA150ACPHi
The source annotation is an empty dictionary
Could not find the following reaction ID in iHN637 FA170ACPHi
The source annotation is an empty dictionary
Could not find the following reaction ID in iHN637 CFACPOA2H_Lumped
The source annotation is an empty dictionary
Could not find the following reaction ID in iHN637 FACOAEcyc170i
The source annotation is an empty dictionary
Could not

In [134]:
rxns_without_annotations = [rxn for rxn in model.reactions if rxn.annotation == {} and 'EX_' not in rxn.id]
len(rxns_without_annotations)

22

In [135]:
model.reactions.CH4trpp.annotation = {u'metanetx.reaction': u'MNXR1138',u'biocyc': u'META:TRANS-RXN0-526'}

In [136]:
model.reactions.DES_9i_Modified.annotation = {u'metanetx.reaction': u'MNXR68672',u'brenda':u'1.14.19.1'}

In [137]:
model.reactions.DES_9i_Modified.id = 'DESAT14_Modified'

In [138]:
model.reactions.CLPNS_MC.annotation = {u'metanetx.reaction': u'MNXR68550'}

In [139]:
model.reactions.PPTGS_MC.annotation = {u'metanetx.reaction': u'MNXR69779'}

In [140]:
model.reactions.OBDHc.annotation = {u'metanetx.reaction': u'MNXR3937',u'biocyc': u'META:RXN-7790',u'rhea':[u'33223', u'33224', u'33225', u'33226']}

In [141]:
model.reactions.UREASE_GAPFILLING.annotation = {u'metanetx.reaction': u'MNXR5299',u'upa': [u'UPA:UCR00774', u'UPA:UER00371'], u'brenda' : u'6.3.4.6',u'kegg.reaction': u'R00774', u'rhea':[u'20896',u'20897',u'20898',u'20899']}

In [142]:
model.reactions.NH3tpp.annotation = {u'metanetx.reaction': u'MNXR82602'}

In [143]:
model.reactions.PFK_ppi.annotation = {u'metanetx.reaction': u'MNXR83628', u'brenda': u'2.7.1.90',u'kegg.reaction':u'R00764'}

In [144]:
model.reactions.PFK_3_ppi.annotation = {u'metanetx.reaction': u'MNXR4174', u'brenda': u'2.7.1.11',u'kegg.reaction':u'R01843', u'biocyc':u'META:RXN0-6541',u'rhea':[u'31443', u'31444', u'31445', u'31446']}

In [145]:
rxns_without_annotations = [rxn for rxn in model.reactions if rxn.annotation == {} and 'EX_' not in rxn.id]
len(rxns_without_annotations)

14

In [146]:
rxns_without_annotations

[<Reaction BIOMASS_Mcapsulatus at 0x114034950>,
 <Reaction 3OAR150_Lumped at 0x11405bc90>,
 <Reaction 3OAR170_Lumped at 0x11405bf50>,
 <Reaction FA150ACPHi at 0x114061450>,
 <Reaction FA170ACPHi at 0x1140616d0>,
 <Reaction CFACPOA2H_Lumped at 0x114061d10>,
 <Reaction FACOAEcyc170i at 0x114061dd0>,
 <Reaction PCOATAi at 0x11406e6d0>,
 <Reaction DM_4hba_c at 0x114076390>,
 <Reaction MC_Average_FattyAcid at 0x114076550>,
 <Reaction MC_AFAA at 0x114076610>,
 <Reaction DM_doxopa_c at 0x11408bd10>,
 <Reaction HAMtpp at 0x114091790>,
 <Reaction NO2tpp at 0x114091850>]

## Metabolite Annotations

In [147]:
mets_without_annotations = [met for met in model.metabolites if met.annotation == {}]
len(mets_without_annotations)

40

In [148]:
mets_without_annotations

[<Metabolite ttdca_c at 0x1140767d0>,
 <Metabolite ptdca_c at 0x114076750>,
 <Metabolite ttdcea_c at 0x1140766d0>,
 <Metabolite hpdca_c at 0x114076690>,
 <Metabolite cpoa2h_c at 0x114076710>,
 <Metabolite pc_MC_c at 0x11403b3d0>,
 <Metabolite pe_MC_c at 0x11403b990>,
 <Metabolite pg_MC_c at 0x114076ad0>,
 <Metabolite clpn_MC_c at 0x114076a90>,
 <Metabolite peptido_MC_c at 0x114076d90>,
 <Metabolite colipa_e at 0x11408ba50>,
 <Metabolite fe2_p at 0x11405b110>,
 <Metabolite tdecoa_c at 0x11405b6d0>,
 <Metabolite tdcoa_c at 0x11405b610>,
 <Metabolite ppACP_c at 0x11406e9d0>,
 <Metabolite ptdcalACP_c at 0x114061790>,
 <Metabolite hpdcalACP_c at 0x114061990>,
 <Metabolite cpoa2hcoa_c at 0x114068050>,
 <Metabolite 2ombzl_c at 0x1140686d0>,
 <Metabolite Ssq23epx_c at 0x11406e050>,
 <Metabolite codhpre6_c at 0x11406ef90>,
 <Metabolite thrp_c at 0x114076490>,
 <Metabolite 4hba_c at 0x114076510>,
 <Metabolite mc_fattyacid_c at 0x114076a10>,
 <Metabolite mc_fattyacidcoa_c at 0x1140768d0>,
 <Metab

In [149]:
model.metabolites.get_by_id('2ombzl_c')

Id,2ombzl_c
Name,"2-Octaprenyl-6-methoxy-1,4-benzoquinol"
Formula,C47H72O3


In [150]:
ecoli.metabolites.get_by_id('2ombzl_c').annotation

{'SBO': 'SBO:0000247',
 'bigg.metabolite': '2ombzl',
 'biocyc': 'META:OCTAPRENYL-METHOXY-BENZOQUINONE',
 'chebi': 'CHEBI:60655',
 'metanetx.chemical': 'MNXM90444',
 'seed.compound': 'cpd15359'}

In [151]:
for met in mets_without_annotations:
    source = ''
    
    query_id = met.id
    
    try:
        source = ecoli.metabolites.get_by_id(query_id)
    except:
        print 'Could not find the following metabolite ID in iJO2366 {}'.format(query_id)
        
    if source != '' and source.annotation != {}:
            for key in source.annotation:
                if key not in met.annotation:
                    met.annotation[key] = source.annotation[key]
    else:
        print 'The source annotation is an empty dictionary'


Could not find the following metabolite ID in iJO2366 ptdca_c
The source annotation is an empty dictionary
Could not find the following metabolite ID in iJO2366 hpdca_c
The source annotation is an empty dictionary
Could not find the following metabolite ID in iJO2366 cpoa2h_c
The source annotation is an empty dictionary
Could not find the following metabolite ID in iJO2366 pc_MC_c
The source annotation is an empty dictionary
Could not find the following metabolite ID in iJO2366 pe_MC_c
The source annotation is an empty dictionary
Could not find the following metabolite ID in iJO2366 pg_MC_c
The source annotation is an empty dictionary
Could not find the following metabolite ID in iJO2366 clpn_MC_c
The source annotation is an empty dictionary
Could not find the following metabolite ID in iJO2366 peptido_MC_c
The source annotation is an empty dictionary
Could not find the following metabolite ID in iJO2366 ppACP_c
The source annotation is an empty dictionary
Could not find the following 

In [152]:
mets_without_annotations = [met for met in model.metabolites if met.annotation == {}]
len(mets_without_annotations)

27

In [153]:
for met in mets_without_annotations:
    source = ''
    
    query_id = met.id
    
    try:
        source = ljungdahlii.metabolites.get_by_id(query_id)
    except:
        print 'Could not find the following metabolite ID in iRC1080 {}'.format(query_id)
        
    if source != '' and source.annotation != {}:
            for key in source.annotation:
                if key not in met.annotation:
                    met.annotation[key] = source.annotation[key]
    else:
        print 'The source annotation is an empty dictionary'


Could not find the following metabolite ID in iRC1080 ptdca_c
The source annotation is an empty dictionary
Could not find the following metabolite ID in iRC1080 hpdca_c
The source annotation is an empty dictionary
Could not find the following metabolite ID in iRC1080 cpoa2h_c
The source annotation is an empty dictionary
Could not find the following metabolite ID in iRC1080 pc_MC_c
The source annotation is an empty dictionary
Could not find the following metabolite ID in iRC1080 pe_MC_c
The source annotation is an empty dictionary
Could not find the following metabolite ID in iRC1080 pg_MC_c
The source annotation is an empty dictionary
Could not find the following metabolite ID in iRC1080 clpn_MC_c
The source annotation is an empty dictionary
Could not find the following metabolite ID in iRC1080 peptido_MC_c
The source annotation is an empty dictionary
Could not find the following metabolite ID in iRC1080 ppACP_c
The source annotation is an empty dictionary
Could not find the following 

In [154]:
mets_without_annotations = [met for met in model.metabolites if met.annotation == {}]
len(mets_without_annotations)

25

In [155]:
model.metabolites.pme_c.id = 'pme_MC_c'

In [156]:
model.metabolites.pdme_c.id = 'pdme_MC_c'

In [157]:
subtilis_file = '/Users/clie/Downloads/iYO844.xml'
subtilis = load_model(subtilis_file)

In [158]:
mets_without_annotations = [met for met in model.metabolites if met.annotation == {}]
len(mets_without_annotations)

25

In [159]:
for met in mets_without_annotations:
    source = ''
    
    query_id = met.id
    
    try:
        source = subtilis.metabolites.get_by_id(query_id)
    except:
        print 'Could not find the following metabolite ID in iYO844 {}'.format(query_id)
        
    if source != '' and source.annotation != {}:
            for key in source.annotation:
                if key not in met.annotation:
                    met.annotation[key] = source.annotation[key]
    else:
        print 'The source annotation is an empty dictionary'


Could not find the following metabolite ID in iYO844 cpoa2h_c
The source annotation is an empty dictionary
Could not find the following metabolite ID in iYO844 pc_MC_c
The source annotation is an empty dictionary
Could not find the following metabolite ID in iYO844 pe_MC_c
The source annotation is an empty dictionary
Could not find the following metabolite ID in iYO844 pg_MC_c
The source annotation is an empty dictionary
Could not find the following metabolite ID in iYO844 clpn_MC_c
The source annotation is an empty dictionary
Could not find the following metabolite ID in iYO844 peptido_MC_c
The source annotation is an empty dictionary
Could not find the following metabolite ID in iYO844 ppACP_c
The source annotation is an empty dictionary
Could not find the following metabolite ID in iYO844 ptdcalACP_c
The source annotation is an empty dictionary
Could not find the following metabolite ID in iYO844 hpdcalACP_c
The source annotation is an empty dictionary
Could not find the following m

In [160]:
mets_without_annotations = [met for met in model.metabolites if met.annotation == {}]
len(mets_without_annotations)

23

In [161]:
model.metabolites.cpoa2h_c.annotation = {u'inchi': u'InChI=1S/C17H32O2/c1-2-3-4-8-11-15-14-16(15)12-9-6-5-7-10-13-17(18)19/h15-16H,2-14H2,1H3,(H,18,19)'}

In [162]:
model.metabolites.get_by_id('1agpgafa_c').id = '1agpgafa_MC_c'

In [163]:
model.metabolites.get_by_id('4hba_c').annotation = {u'bigg.metabolite': u'4hba',
 u'biocyc': u'META:4-HYDROXY-BENZYL-ALCOHOL',
 u'chebi': u'CHEBI:67410',
 u'kegg.compound': u'C17467',
 u'metanetx.chemical': u'MNXM107505',
 u'seed.compound': [u'cpd15378', u'cpd17595']}

In [164]:
model.metabolites.doxopa_c.annotation = {u'metanetx.chemical': u'MNXM148580'}

## Add proper transport reactions for relevant metabolites

In [165]:
model.change_objective(model.reactions.BIOMASS_Mcapsulatus)

In [166]:
model.medium

,reaction_id,reaction_name,lower_bound,upper_bound
0,EX_pi_c,phosphate exchange,-1000.00,1000.0
1,EX_o2_c,dioxygen exchange,-1000.00,1000.0
2,EX_nh3_c,NH4(+) exchange,-1000.00,1000.0
3,EX_so4_c,sulfate exchange,-1000.00,1000.0
4,EX_cl_c,Cl(-) exchange,-1000.00,1000.0
5,EX_na1_c,Na(+) exchange,-1.00,1000.0
6,EX_k_c,K(+) exchange,-1000.00,1000.0
7,EX_mg2_c,Mg(2+) exchange,-1000.00,1000.0
8,EX_ca2_c,Ca(2+) exchange,-1000.00,1000.0
9,EX_ch4_p,methane exchange,-18.46,-0.1


In [167]:
solution = model.solve()

In [168]:
model.reactions.EX_rdmbzi_c.id = 'DM_rdmbzi_c'

In [169]:
relevant_exchanges = [rxn for rxn in model.reactions if rxn.id.startswith('EX_') and abs(rxn.flux) != 0]

In [173]:
relevant_exchanges.pop(4)

Id,EX_ACP_c
Name,holo-[acyl-carrier protein] exchange
Stoichiometry,ACP_c -->
GPR,
Lower bound,0.000000
Upper bound,1000.000000


In [174]:
len(relevant_exchanges)

18

In [175]:
exchanges_to_be_removed = []

for rxn in model.reactions:
    if rxn.id.startswith('EX_') and rxn not in relevant_exchanges:
        exchanges_to_be_removed.append(rxn)
        

In [176]:
model.remove_reactions(exchanges_to_be_removed)

In [177]:
solution = model.solve()

In [178]:
[rxn for rxn in model.reactions if rxn.id.startswith('EX_')]

[<Reaction EX_h_c at 0x114001150>,
 <Reaction EX_co2_c at 0x1140014d0>,
 <Reaction EX_h2o_c at 0x1140018d0>,
 <Reaction EX_pi_c at 0x114001ad0>,
 <Reaction EX_o2_c at 0x113b28250>,
 <Reaction EX_nh3_c at 0x113b3aa10>,
 <Reaction EX_so4_c at 0x113ad4050>,
 <Reaction EX_h2o2_c at 0x113ae5410>,
 <Reaction EX_co_c at 0x113af72d0>,
 <Reaction EX_cl_c at 0x113506fd0>,
 <Reaction EX_na1_c at 0x114034750>,
 <Reaction EX_k_c at 0x1140347d0>,
 <Reaction EX_mg2_c at 0x114034850>,
 <Reaction EX_ca2_c at 0x1140348d0>,
 <Reaction EX_ch4_p at 0x114041d90>,
 <Reaction EX_fe2_p at 0x114054e50>,
 <Reaction EX_cobalt2_c at 0x11406e8d0>,
 <Reaction EX_cu_c at 0x114098a90>]

In [188]:
from cameo.util import TimeMachine

In [191]:
from cameo.exceptions import Infeasible

In [203]:
def test_autonomous_metabolite_production(model):
    with TimeMachine() as tm:
        producable_from_nothing = []    
        for rxn in model.reactions:
            
            if rxn.id.startswith("DM_") or  rxn.id.startswith("EX_"):
                rxn.change_bounds(0,0,tm)
                
        for met in model.metabolites:
            
            try:
                ex_rxn = model.reactions.get_by_id('EX_' + met.id)
                ex_rxn.change_bounds(0,1000,tm)
                model.objective = ex_rxn
            except:
                model.add_demand(met,'TestDM_',tm)
                ex_rxn = model.reactions.get_by_id('TestDM_' + met.id)
                ex_rxn.change_bounds(0,1000,tm)
                model.objective = ex_rxn
                
            with pytest.raises(Infeasible):
                sol = model.solve()
                if sol.f != 0:
                    producable_from_nothing.append(met.id)
        
        return producable_from_nothing
                

In [204]:
producable_from_nothing = test_autonomous_metabolite_production(model)

In [205]:
len(producable_from_nothing)

0

In [213]:
import cobra

target_filename_json = relative_directory + '/Reconstructions/MethylococcusModel12.json'
target_filename_xml = relative_directory + '/Reconstructions/MethylococcusModel12.xml'
cobra.io.write_sbml_model(model, target_filename_xml, use_fbc_package=True)
cobra.io.save_json_model(model, target_filename_json,pretty = True)

In [ ]:
import escher
        
maps = escher.Builder(
                    model=model,
                    map_name = 'Methylococcus capsulatus genome-scale metabolic model',
                    map_json='/Users/clie/Desktop/EFPro2/Reconstructions/Metabolic Map/16UpdateDuplicateAndParallelReactions.json',
                    show_gene_reaction_rules=True,
                    use_3d_transform = True
                    )

maps.set_reaction_scale([ { 'type': 'min', 'color': 'rgb(235,89,162)', 'size': 12 },
   { 'type': 'median', 'color': 'rgb(200,200,200)', 'size': 20 },
   { 'type': 'max', 'color': 'rgb(255,255,109)', 'size': 25 } ])
maps.display_in_browser()